In [1]:
!pip install monetdblite
import monetdblite as m
import pandas as pd
from datetime import datetime
m.init('/tmp/MonetDB Database')

     |████████████████████████████████| 7.3MB 3.8MB/s 


In [2]:
!wget -O Robust04.rar https://dl.dropboxusercontent.com/s/yyug8pi399qi2s9/Robust04%2Bpos%2Btf.rar?dl=0
!unrar e -o+ Robust04.rar

--2019-11-14 09:01:24--  https://dl.dropboxusercontent.com/s/yyug8pi399qi2s9/Robust04%2Bpos%2Btf.rar?dl=0
Resolving dl.dropboxusercontent.com (dl.dropboxusercontent.com)... 162.125.8.6, 2620:100:601b:6::a27d:806
Connecting to dl.dropboxusercontent.com (dl.dropboxusercontent.com)|162.125.8.6|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 459824653 (439M) [application/rar]
Saving to: ‘Robust04.rar’

Robust04.rar        100%[===================>] 438.52M  54.9MB/s    in 8.3s    

2019-11-14 09:01:33 (52.6 MB/s) - ‘Robust04.rar’ saved [459824653/459824653]


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from Robust04.rar

Extracting  dict.csv                                                       0%  OK 
Extracting  docs.csv                                                       0%  1%  OK 
Extracting  terms.csv                                                      1%  2%  3%  4%  5%  6%

In [0]:
m.sql("CREATE TABLE dict(termid INTEGER "
                       ",term   VARCHAR(99) "
                       ",df     INTEGER)")
m.sql("CREATE TABLE terms(termid INTEGER "
                        ",docid  INTEGER "
                        ",pos    INTEGER "
                        ",tf     INTEGER)")
m.sql("CREATE TABLE docs(name  VARCHAR(99) "
                       ",docid INTEGER "
                       ",len   INTEGER "
                       ",temp  INTEGER)") 

In [0]:
m.sql("COPY INTO dict "
      "FROM '/content/dict.csv' "
      "USING DELIMITERS '|'")
m.sql("COPY INTO terms "
      "FROM '/content/terms.csv' "
      "USING DELIMITERS '|'")
m.sql("COPY INTO docs "
      "FROM '/content/docs.csv' "
      "USING DELIMITERS '|'")

In [0]:
def make_query(*args: str):
    """Function to make query table in database.
    
    Args:
        args: concatenation of strings to be made into a query
        
    Returns [str]:
        Number of terms in the query as string.
    """
    query = "('" + args[0] + "')"
    for arg in args[1:]:
        query += ", ('" + arg + "')"
    
    m.sql("DROP TABLE IF EXISTS query")
    m.sql("CREATE TABLE query(term VARCHAR(99))")
    m.sql("INSERT INTO query VALUES " + query)     
        
    return str(len(args))

In [19]:
len_query = make_query('wizard', 'hat')

cur_time = datetime.now()

qterms = m.sql("WITH qtermids "
                    "AS (SELECT dict.termid "
                              ",dict.df "
                        "FROM dict "
                        "JOIN query "
                        "ON dict.term = query.term"
                        ") "
               "SELECT terms.termid "
                     ",terms.docid "
                     ",terms.pos "
                     ",terms.tf "
                     ",qtermids.df "
               "FROM terms "
               "JOIN qtermids "
               "ON terms.termid = qtermids.termid")

time_delta = datetime.now() - cur_time
print('Query time: {}'.format(time_delta))

pd.DataFrame.from_dict(qterms)

Query time: 0:00:00.192979


,termid,docid,pos,tf,df
0,434301,44,589,1,1718
1,434301,152,497,1,1718
2,434301,281,92,1,1718
3,434301,339,1024,1,1718
4,434301,396,696,1,1718
...,...,...,...,...,...
2590,694497,323249,53,2,269
2591,694497,323249,186,2,269
2592,694497,324788,1341,1,269
2593,694497,326675,155,1,269


In [18]:
len_query = make_query('new', 'york')

N = m.sql("SELECT COUNT(*) "
          "FROM docs")
N = str(pd.DataFrame.from_dict(N).iloc[0, 0])
avgdl = m.sql("SELECT AVG(docs.len) "
              "FROM docs")
avgdl = str(pd.DataFrame.from_dict(avgdl).iloc[0, 0])

k1 = '1.2'
b = '0.75'

k = '20'

cur_time = datetime.now()

t = m.sql("WITH qtermids "
               "AS (SELECT dict.termid "
                         ",dict.df "
                   "FROM dict "
                   "JOIN query "
                   "ON dict.term = query.term"
                   ") "
          ", qterms "
               "AS (SELECT terms.termid "
                         ",terms.docid "
                         ",terms.pos "
                         ",terms.tf "
                         ",qtermids.df "
                   "FROM terms "
                   "JOIN qtermids "
                   "ON terms.termid = qtermids.termid"
                   ") "
          ", condocs "
               "AS (SELECT qterms.docid "
                   "FROM qterms "
                   "GROUP BY qterms.docid "
                   "HAVING COUNT(DISTINCT qterms.termid) = " + len_query +
                   ") "
          ", okapisubscores "
               "AS (SELECT qterms.docid "
                         ",( LOG((" + N + "-df+0.5)/(df+0.5))*tf*(" + k1 + "+1)"
                             "/"
                            "(tf+" + k1 + "*(1-" + b + "+" + b + "*len/" + avgdl + "))"
                           ") AS subscore "
                   "FROM qterms "
                   "JOIN condocs "
                   "ON qterms.docid = condocs.docid "
                   "JOIN docs "
                   "ON qterms.docid = docs.docid "
                   ") "
          ", topdocs "
               "AS (SELECT okapisubscores.docid "
                         ",( ROW_NUMBER() "
                            "OVER(ORDER BY SUM(okapisubscores.subscore) DESC)"
                           ") AS row "
                   "FROM okapisubscores "
                   "GROUP BY okapisubscores.docid "
                   ") "
          ", pairs "
               "AS (SELECT qterms1.termid AS termid1 "
                         ",qterms2.termid AS termid2 "
                         ",qterms1.docid "
                         ",1.0/(qterms1.pos-qterms2.pos) AS tpi "
                         ",( CASE WHEN qterms1.df > qterms2.df THEN qterms1.df "
                                 "ELSE qterms2.df "
                            "END"
                           ") AS maxdf "
                   "FROM qterms AS qterms1 "
                  #  "JOIN topdocs "
                  #  "ON qterms1.docid = topdocs.docid AND "
                      # "topdocs.row BETWEEN 1 AND " + k +
                   "JOIN qterms AS qterms2 "
                   "ON qterms1.docid = qterms2.docid AND "
                      "NOT qterms1.termid = qterms2.termid AND "
                      "qterms1.pos - qterms2.pos BETWEEN 1 AND 5"
                   ") "
          ",tpisums "
               "AS (SELECT pairs.termid1 "
                         ",pairs.termid2 "
                         ",pairs.docid "
                         ",SUM(pairs.tpi) AS tpisum "
                   "FROM pairs "              
                   "GROUP BY pairs.termid1 "
                           ",pairs.termid2 "
                           ",pairs.docid"
                   ") "
          ", subscores "
               "AS (SELECT pairs.docid "
                         ",( LOG((" + N + "-maxdf+0.5)/(maxdf+0.5))*tpisum*(" + k1 + "+1)"
                             "/"
                            "(tpisum+" + k1 + "*(1-" + b + "+" + b + "*len/" + avgdl + "))"
                           ") AS subscore "
                   "FROM pairs "
                   "JOIN tpisums "
                   "ON pairs.termid1 = tpisums.termid1 AND "
                      "pairs.termid2 = tpisums.termid2 AND "
                      "pairs.docid = tpisums.docid "
                   "JOIN docs "
                   "ON pairs.docid = docs.docid"
                   ") "
          ", scores "
               "AS (SELECT subscores.docid "
                         ",SUM(subscores.subscore) AS score "
                   "FROM subscores "
                   "GROUP BY subscores.docid"
                   ") "
          "SELECT scores.docid "
                ",scores.score "
          "FROM scores "
          "ORDER BY scores.score DESC ")

time_delta = datetime.now() - cur_time
print('Query time: {}'.format(time_delta))

pd.DataFrame.from_dict(t)

Query time: 0:00:00.599279


,docid,score
0,50961,128.848305
1,84012,105.641038
2,53470,97.747231
3,134713,68.253371
4,18841,52.145487
...,...,...
19465,108850,0.041620
19466,19485,0.038295
19467,111744,0.028130
19468,241516,0.023688
